# 1. load the keypoints to fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.utils as fou
import os

# Assuming you have a list of image paths and corresponding TXT file paths
image_paths = [...]  # List of image file paths
txt_paths = [...]  # List of corresponding TXT file paths for each image

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_pose_estimation")
def parse_pose_estimation(txt_file):
    """
    Parse the pose estimation data from a TXT file.

    Parameters:
    txt_file (str): Path to the TXT file.

    Returns:
    list: A list of pose estimation data (each line parsed into a list of values).
    """
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Loop over images and corresponding TXT files
for img_path, txt_path in zip(image_paths, txt_paths):
    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(txt_path)
    
    # List to hold all detections and keypoints for the current image
    detections = []
    
    for pose in pose_estimations:
        # Extract and scale bounding box and keypoints
        x_center_scaled = pose[1] * 5312
        y_center_scaled = pose[2] * 2988
        width_scaled = pose[3] * 5312
        height_scaled = pose[4] * 2988

        keypoints = []
        for i in range(5, len(pose), 3):
            x_kp_scaled = pose[i] * 5312
            y_kp_scaled = pose[i + 1] * 2988
            confidence = pose[i + 2]
            keypoints.append(fo.Keypoint(point=[x_kp_scaled, y_kp_scaled], confidence=confidence))

        bounding_box = [
            (x_center_scaled - width_scaled / 2) / 5312,
            (y_center_scaled - height_scaled / 2) / 2988,
            width_scaled / 5312,
            height_scaled / 2988
        ]

        # Create a detection object
        detection = fo.Detection(label="prawn", bounding_box=bounding_box, keypoints=keypoints)
        detections.append(detection)

    # Create a sample for FiftyOne
    sample = fo.Sample(filepath=img_path)
    sample["detections"] = fo.Detections(detections=detections)
    dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)




# 2. load metadata from excel file

In [ ]:
import pandas as pd
import fiftyone as fo

# Read the Excel file
file_path = 'your_file_path.xlsx'  # Replace with your actual file path
metadata_df = pd.read_excel(file_path)

# Display the first few rows to ensure it's loaded correctly
print(metadata_df.head())


# Assuming you have a list of image paths
image_paths = [...]  # List of image file paths

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_metadata")

# Loop through each image and add metadata
for img_path in image_paths:
    # Extract the filename without extension
    filename = img_path.split('/')[-1].replace('.jpg', '')  # Adjust if not .jpg
    
    # Match the filename with the metadata
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        
        # Create a sample and attach metadata
        sample = fo.Sample(filepath=img_path)
        
        # Add each metadata field to the sample
        for key, value in metadata.items():
            if key != 'file name':  # Exclude the file name itself
                sample[key] = value
        
        # Add the sample to the dataset
        dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)


# 3.bounding rectangle between detection and imagej

In [ ]:
import pandas as pd

# Load ImageJ bounding rectangles from a CSV file
imagej_bboxes_df = pd.read_csv('imagej_bboxes.csv')

# Example structure of `imagej_bboxes_df`:
# Columns: ['file name', 'x1', 'y1', 'x2', 'y2']
def calculate_bbox_distance(detection_bbox, imagej_bbox):
    """
    Calculate the Euclidean distance between the centers of two bounding boxes.

    Parameters:
    detection_bbox (tuple): (x1, y1, x2, y2) coordinates of the detection bounding box.
    imagej_bbox (tuple): (x1, y1, x2, y2) coordinates of the ImageJ bounding box.

    Returns:
    float: Euclidean distance between the centers of the bounding boxes.
    """
    # Calculate centers
    detection_center = ((detection_bbox[0] + detection_bbox[2]) / 2, (detection_bbox[1] + detection_bbox[3]) / 2)
    imagej_center = ((imagej_bbox[0] + imagej_bbox[2]) / 2, (imagej_bbox[1] + imagej_bbox[3]) / 2)
    
    # Calculate Euclidean distance between centers
    distance = np.sqrt((detection_center[0] - imagej_center[0])**2 + (detection_center[1] - imagej_center[1])**2)
    return distance
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    
    # Assume `detections` is a list of bounding boxes from the model
    detection_bboxes = [...]  # Replace with actual detection bounding boxes
    
    # Match with ImageJ bounding box
    imagej_bbox = imagej_bboxes_df[imagej_bboxes_df['file name'] == filename].iloc[0]
    imagej_bbox = (imagej_bbox['x1'], imagej_bbox['y1'], imagej_bbox['x2'], imagej_bbox['y2'])
    
    distances = []
    for detection_bbox in detection_bboxes:
        distance = calculate_bbox_distance(detection_bbox, imagej_bbox)
        distances.append(distance)
    
    # Create a FiftyOne sample and add metadata
    sample = fo.Sample(filepath=img_path)
    sample['bbox_distances'] = distances  # Store all distances for this image
    
    dataset.add_sample(sample)

# Launch FiftyOne app to visualize
session = fo.launch_app(dataset)


# 4. calculate length based on pinhole model

In [ ]:
import numpy as np

def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    """
    Calculate the real-life width of an object.

    Parameters:
    focal_length (float): Focal length of the camera lens in millimeters (mm).
    distance_to_object (float): Distance from the camera to the object in millimeters (mm).
    width_in_pixels (int): Width of the object in pixels on the image sensor.
    pixel_size (float): Size of a pixel on the image sensor in millimeters (mm).

    Returns:
    float: Real-life width of the object in centimeters (cm).
    """
    # Calculate the width of the object in the image sensor plane in millimeters
    width_in_sensor = width_in_pixels * pixel_size

    # Calculate the real-life width of the object using the similar triangles principle
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length

    # Convert the width from millimeters to centimeters
    real_width_cm = real_width_mm / 10.0

    return real_width_cm

def calculate_euclidean_distance(keypoint1, keypoint2):
    """
    Calculate the Euclidean distance between two keypoints.

    Parameters:
    keypoint1, keypoint2: Tuples of (x, y) coordinates of the keypoints in pixels.

    Returns:
    float: Euclidean distance between the keypoints in pixels.
    """
    return np.sqrt((keypoint1[0] - keypoint2[0])**2 + (keypoint1[1] - keypoint2[1])**2)

# Example of how to integrate this with your FiftyOne dataset
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        sample = fo.Sample(filepath=img_path)
        
        # Add metadata
        for key, value in metadata.items():
            if key != 'file name':
                sample[key] = value
        
        # Calculate the real height or width if keypoints and necessary metadata are available
        if 'height(mm)' in metadata:
            height_mm = metadata['height(mm)']
            focal_length = 6.82  # Example focal length in mm
            pixel_size = 0.0014  # Example pixel size in mm
            keypoints = [...]  # Replace with actual keypoints from your data
            
            if len(keypoints) >= 2:
                # Calculate the Euclidean distance in pixels
                euclidean_distance_pixels = calculate_euclidean_distance(keypoints[0], keypoints[1])
                
                # Calculate the real width/height in centimeters
                real_width_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
                
                # Attach the calculated real width/height to the sample
                sample["real_width_cm"] = real_width_cm
        
        dataset.add_sample(sample)

session = fo.launch_app(dataset)


# complete maybe

In [53]:
datasets = fo.list_datasets()

for dataset_name in datasets:
    if dataset_name.startswith('prawn_combined_dataset'):
        fo.delete_dataset(dataset_name)


Could not connect session, trying again in 10 seconds



In [ ]:
import fiftyone as fo
import pandas as pd
import numpy as np
import os
import ast 
from tqdm import tqdm
import json


# Load the existing filtered data with PrawnIDs
filtered_data_file_path = r'C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\Filtered_Data.csv'  # Replace with your actual file path
filtered_df = pd.read_csv(filtered_data_file_path)

# Load the additional metadata from another Excel file
metadata_file_path = r"C:\Users\gbo10\OneDrive\research\thesis and paper\test images.xlsx"  # Replace with your actual file path
metadata_df = pd.read_excel(metadata_file_path)

# Function to parse pose estimation data from a TXT file
def parse_pose_estimation(txt_file):
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Function to calculate Euclidean distance between keypoints
def calculate_euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to calculate the real-life width/height
def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    width_in_sensor = width_in_pixels * pixel_size
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length
    return real_width_mm
def extract_identifier_from_gt(filename):
    return filename.split('-')[0]

folder_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\car'  # Replace with the folder containing the images and TXT files

# Assuming you have the following lists
image_paths = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

prediction_folder_path=r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

txt_paths = [os.path.join(prediction_folder_path, txt) for txt in os.listdir(prediction_folder_path) if txt.endswith('.txt')]
# Create a FiftyOne dataset
ground_truth_paths = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\labels"  # Replace with the folder containing the ground truth TXT files

ground_truth_paths_text = [os.path.join(ground_truth_paths, txt) for txt in os.listdir(ground_truth_paths) if txt.endswith('.txt')]


import fiftyone as fo

# List all datasets in FiftyOne

# Loop through the list and delete datasets that match the condition


#delete all datasets


# Create a new dataset

dataset = fo.Dataset("prawn_combined_dataset22", overwrite=True)

dataset.default_skeleton = fo.KeypointSkeleton(
    labels=[
        "start_carapace",
        "eyes",
    ],
    edges=[
        [0, 1],  # Connect keypoint 0 to keypoint 1  # Connect keypoint 1 to keypoint 2
        # Add more connections as needed
    ],
)
# Loop over images and corresponding TXT files
for img_path in image_paths:

    # Extract the relevant part for matching
    filename = os.path.splitext(os.path.basename(img_path))[0] 
    parts = filename.split('_')
    relevant_part = f"{parts[1][-3:]}_{parts[3].split('.')[0]}"
     # e.g., undistorted_GX010152_36_378.jpg_gamma
    identifier = filename.replace('undistorted_', '').replace('.jpg_gamma', '')  # Extract the identifier from the filename


    # Construct the paths to the prediction and ground truth files
    prediction_txt_path = os.path.join(prediction_folder_path, f"{filename}.txt")

    # Match ground truth based on the extracted identifier
    ground_truth_txt_path = None
    for gt_file in ground_truth_paths_text:
        b= extract_identifier_from_gt(os.path.basename(gt_file))
        if b == identifier:
            ground_truth_txt_path = gt_file

            break

    
    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(prediction_txt_path)
    
    ground_truths = parse_pose_estimation(ground_truth_txt_path)

    
    keypoints_list = []  # List to store keypoints
    detections = []  # List to store detection bounding boxes
    ground_truth_keypoints_list = []
    ground_truth_detection_list = []

    for pose in pose_estimations:
        if len(pose)==11:        

            x1_rel = pose[1] 
            y1_rel = pose[2] 
            width_rel = pose[3] 
            height_rel = pose[4] 

            
            #center to top left
            x1_rel = x1_rel - width_rel/2
            y1_rel = y1_rel - height_rel/2
        # Calc



            keypoints = []
            confidences = []
            for i in range(5, len(pose), 3):
                x_kp_scaled = pose[i] 
                y_kp_scaled = pose[i + 1] 
                confidence = pose[i + 2]
                keypoints.append([x_kp_scaled, y_kp_scaled])
            

            confidences = [float(pose[i+2]) for i in range(5, len(pose), 3)]


         
                
            keypoint=fo.Keypoint(points=keypoints, confidence=None)
            keypoints_list.append(keypoint)


            keypoints_dict= {'point1':keypoints[0],'point2':keypoints[1]}
            detections.append(fo.Detection
                              (label="prawn", bounding_box=[x1_rel, y1_rel, width_rel, height_rel],
                               attributes={'keypoints':keypoints_dict}))
            
    for gt_pose in ground_truths:
            x1_rel = gt_pose[1]
            y1_rel = gt_pose[2]
            width_rel = gt_pose[3]
            height_rel = gt_pose[4]

            # Convert center to top-left
            x1_rel = x1_rel - width_rel / 2
            y1_rel = y1_rel - height_rel / 2

            keypoints = []
            for i in range(5, len(gt_pose), 3):
                x_kp_scaled = gt_pose[i]
                y_kp_scaled = gt_pose[i + 1]
                keypoints.append([x_kp_scaled, y_kp_scaled])

            ground_truth_keypoints_list.append(fo.Keypoint(points=keypoints))
            ground_truth_detection_list.append(fo.Detection(label="prawn_truth", bounding_box=[x1_rel, y1_rel, width_rel, height_rel]))

    sample = fo.Sample(filepath=img_path)
    
    sample["ground_truth_keypoints"] = fo.Keypoints(keypoints=ground_truth_keypoints_list)


    sample["ground_truth_detections"] = fo.Detections(detections=detections)   
    # Add keypoints as a separate field
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints_list)
   
    sample["detections_predictions"] = fo.Detections(detections=detections)
    


    
    
    # Match the filename with the corresponding rows in the filtered data
    matching_rows = filtered_df[filtered_df['Label'] == f'carapace:{filename}']
    
    # Add metadata from the additional Excel file
    metadata_row = metadata_df[metadata_df['file name'] == relevant_part]

    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict() 
        for key, value in metadata.items():
            if key != 'file name': 
                sample[key] = value
            
                        
    else:
        print(f"No metadata found for {relevant_part}")
    true_detections=[]

    for _, row in matching_rows.iterrows():
        prawn_id = row['PrawnID']
        prawn_bbox = ast.literal_eval(row['BoundingBox_1'])  # Replace with the correct column name

        # Convert the tuple elements to floats
        prawn_bbox = tuple(float(coord) for coord in prawn_bbox)# Replace with correct bounding box columns
        
        prawn_normalized_bbox=[prawn_bbox[0]/5312, prawn_bbox[1]/2988, prawn_bbox[2]/5312, prawn_bbox[3]/2988]

        # Convert PrawnID bounding box to absolute coordinates
        
        
        true_detections.append(fo.Detection(label="prawn_true", bounding_box=prawn_normalized_bbox))

        prawn_point=(prawn_bbox[0]/5312, prawn_bbox[1]/2988)
        

        # Find the closest detection bounding box within the same image (Label)
        min_distance = float('inf')
        closest_detection = None
        
        for detection_bbox in sample["detections_predictions"].detections:
            detection_bbox_coords = detection_bbox.bounding_box
            
            det_point=(detection_bbox_coords[0], detection_bbox_coords[1])
            
            distance = calculate_euclidean_distance(prawn_point, det_point)
            
            if distance < min_distance:
                min_distance = distance
                closest_detection = detection_bbox
        
        if closest_detection is not None:
            height_mm = sample['heigtht(mm)']  # Use appropriate column for height in mm
            focal_length = 24.22
            pixel_size = 0.00716844 

                

            keypoints_dict2 = closest_detection.attributes["keypoints"]

            keypoints1 = [keypoints_dict2['point1'], keypoints_dict2['point2']]    

            keypoint1_scaled = [keypoints1[0][0] * 5312, keypoints1[0][1] * 2988]
            keypoint2_scaled = [keypoints1[1][0] * 5312, keypoints1[1][1] * 2988]

            euclidean_distance_pixels = calculate_euclidean_distance(keypoint1_scaled, keypoint2_scaled)
            real_length_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
            

            # Update the filtered_df with the calculated lengths
            filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'RealLength(cm)'] = real_length_cm
            
            # Add floating text near the keypoints in FiftyOne

            true_length=filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'Avg_Length'].values[0]


            closest_detection_label =f'MPError: {abs(real_length_cm - true_length) / true_length * 100:.2f}%, true length: {true_length:.2f}cm, pred length: {real_length_cm:.2f}cm'
            
            closest_detection.label=closest_detection_label


            #if mpe>25: tag sample
            if abs(real_length_cm - true_length) / true_length * 100 > 25:
                if "MPE>25" not in sample.tags:
                    sample.tags.append("MPE>25")
            
    # Assuming 'sample' is your FiftyOne sample object
        
    sample["true_detections"] = fo.Detections(detections=true_detections)            

    dataset.add_sample(sample)

ground_truth_count=0

for sample in dataset:
    ground_truth_count+=len(sample["ground_truth_detections"].detections)

print('gt',ground_truth_count)

detections_predictions_count=0

for sample in dataset:
    detections_predictions_count+=len(sample["detections_predictions"].detections)

print('pred',detections_predictions_count)


# Save the updated filtered data with the calculated lengths
# filtered_df.to_excel('updated_filtered_data_with_lengths.xlsx', index=False)

# Launch the FiftyOne app to visualize

# Define a custom view

# Configure the view to show your custom attributes


session = fo.launch_app(dataset, port=5151)


# Launch the app with the custom view



In [27]:
import numpy as np
import cv2
from ultralytics import SAM
from math import sqrt
from random import randint, shuffle
import fiftyone as fo
import fiftyone.core.labels as fol
import os

# Helper Classes and Functions for Circle Calculation
class Point:
    def __init__(self, X=0, Y=0) -> None:
        self.X = X
        self.Y = Y

class Circle:
    def __init__(self, c=Point(), r=0) -> None:    
        self.C = c
        self.R = r

def dist(a, b):
    return sqrt((a.X - b.X) ** 2 + (a.Y - b.Y) ** 2)

def is_inside(c, p):
    return dist(c.C, p) <= c.R

def get_circle_center(bx, by, cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D), (bx * C - cx * B) / (2 * D))

def circle_from1(A, B):
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0)
    return Circle(C, dist(A, B) / 2.0)

def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y, C.X - A.X, C.Y - A.Y)
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))

def is_valid_circle(c, P):
    for p in P:
        if not is_inside(c, p):
            return False
    return True

def min_circle_trivial(P):
    assert len(P) <= 3
    if not P:
        return Circle()
    elif len(P) == 1:
        return Circle(P[0], 0)
    elif len(P) == 2:
        return circle_from1(P[0], P[1])
    for i in range(3):
        for j in range(i + 1, 3):
            c = circle_from1(P[i], P[j])
            if is_valid_circle(c, P):
                return c
    return circle_from2(P[0], P[1], P[2])

def welzl_helper(P, R, n):
    if n == 0 or len(R) == 3:
        return min_circle_trivial(R)
    idx = randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    d = welzl_helper(P, R.copy(), n - 1)
    if is_inside(d, p):
        return d
    R.append(p)
    return welzl_helper(P, R.copy(), n - 1)

def welzl(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))
def yolo_key_to_box(yolo_line, img_width=640, img_height=640):
    # YOLO format: class x_center y_center width height (normalized)
    yolo_data = yolo_line.strip().split()
    x_center, y_center, width, height = map(float, yolo_data[1:5])


    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    # Calculate top-left and bottom-right coordinates
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return np.array([ x_min, y_min, x_max, y_max])

def convert_yolo_key_to_boxes(yolo_file):
    boxes = []
    with open(yolo_file, 'r') as file:
        for line in file:
            box = yolo_key_to_box(line)
            boxes.append(box)
    return boxes

def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None 




# Initialize FiftyOne dataset
dataset = fo.Dataset("prawn_segmentation", overwrite=True)

# Load SAM model
model = SAM(r'mobile_sam.pt')

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

# Iterate over images
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, image_name)
       
        core_name = image_name.split('.')[0]
        # Create the corresponding YOLO file path

                

        yolo_file = find_matching_yolo_file(core_name, yolo_dir)
        
        # Convert YOLO annotations to bounding boxes
        boxes = convert_yolo_key_to_boxes(yolo_file)
        
        # Load image
        image = cv2.imread(image_path)
        
        # Perform segmentation
        
        # Initialize list to store segmentations
        all_polygons = []
        all_attributes = []

        
        # Process each mask individually
        for xy in results[0].masks.xy:
                 
    # Normalize points to [0, 1] range
                normalized_points = [(float(p[0]) / image.shape[1], float(p[1]) / image.shape[0]) for p in xy]
                all_polygons.append(normalized_points)
                points = [Point(p[0], p[1]) for p in xy]
                mec = welzl(points)
                diameter = mec.R * 2 
                all_attributes.append({'diameter': diameter})
                # Convert contour to a format suitable for FiftyOne
                segmentation = fol.Polyline(
                        points=all_polygons,
                        closed=True,
                        filled=True,
                        attributes=all_attributes

                        
                    )
        
        # Add image and segmentation to FiftyOne dataset
        sample = fo.Sample(filepath=image_path)
        sample["segmentations"] =fol.Polylines(polylines=[segmentation])

        dataset.add_sample(sample)

# Save dataset
dataset.save()
session = fo.launch_app(dataset, port=5153)


0: 1024x1024 1617.6ms
Speed: 23.0ms preprocess, 1617.6ms inference, 6.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1602.6ms
Speed: 18.6ms preprocess, 1602.6ms inference, 8.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1428.8ms
Speed: 19.0ms preprocess, 1428.8ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1512.6ms
Speed: 22.0ms preprocess, 1512.6ms inference, 4.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1498.9ms
Speed: 23.9ms preprocess, 1498.9ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1508.2ms
Speed: 22.5ms preprocess, 1508.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1320.1ms
Speed: 19.0ms preprocess, 1320.1ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1388.6ms
Speed: 20.0ms preprocess, 1388.6ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)


RuntimeError: Client is not connected

In [3]:
import os
import cv2
import numpy as np
from ultralytics import SAM
from tqdm import tqdm


# YOLO format conversion
def yolo_key_to_box(yolo_line, img_width=640, img_height=360):
    yolo_data = yolo_line.strip().split()
    x_center, y_center, width, height = map(float, yolo_data[1:5])

    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return np.array([x_min, y_min, x_max, y_max])

def convert_yolo_key_to_boxes(yolo_file):
    boxes = []
    with open(yolo_file, 'r') as file:
        for line in file:
            box = yolo_key_to_box(line)
            boxes.append(box)
    return boxes

def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None 

# Initialize SAM model
model = SAM(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\fifty_one\measurements\mobile_sam.pt')

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict90\labels"
segmenation_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict90\segmentations2_drawn"

os.makedirs(segmenation_dir, exist_ok=True)
# Iterate over images
for image_name in tqdm(os.listdir(image_dir)):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, image_name)

        core_name = image_name.split('.')[0]
        
        # Create the corresponding YOLO file path
        yolo_file = find_matching_yolo_file(core_name, yolo_dir)
        if yolo_file is None:
            print(f"No matching YOLO file found for {image_name}")
            continue
        else:
            print(f"Found matching YOLO file for {image_name}")
        # Convert YOLO annotations to bounding boxes
        boxes = convert_yolo_key_to_boxes(yolo_file)
        
        # Load image
        image = cv2.imread(image_path)
        
        # Perform segmentation
        results = model(image, imgsz=(640,360),save=True, bboxes=np.array(boxes))
        
        results[0].save(filename=os.path.join(segmenation_dir,f'{core_name}.jpg'))
        # Prepare a .txt file to save the segmentations
        output_txt_path = os.path.join(image_dir, f"{core_name}_segmentations.txt")
    


        #taigu_sam



        with open(output_txt_path, 'w') as f:
            # Process each mask individually
            for xy in results[0].masks.xy:
                # Convert the xy coordinates into a string format for saving
                xy_str = " ".join(map(str, xy.flatten().tolist()))  # Flatten and convert to string
                f.write(xy_str + "\n")  # Write each xy as a line
                print(f"Saved segmentation for {image_name} to {output_txt_path}")


100%|██████████| 88.0M/88.0M [01:05<00:00, 1.40MB/s]
  0%|          | 0/60 [00:00<?, ?it/s]

Found matching YOLO file for GX010152_36_378-jpg_gamma_jpg.rf.9acbfb13ae54b206727a686e4c1c7953.jpg

0: 1024x1024 3262.1ms
Speed: 37.3ms preprocess, 3262.1ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\segment\predict58


  2%|▏         | 1/60 [00:05<05:45,  5.85s/it]

Saved segmentation for GX010152_36_378-jpg_gamma_jpg.rf.9acbfb13ae54b206727a686e4c1c7953.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010152_36_378-jpg_gamma_jpg_segmentations.txt
Saved segmentation for GX010152_36_378-jpg_gamma_jpg.rf.9acbfb13ae54b206727a686e4c1c7953.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010152_36_378-jpg_gamma_jpg_segmentations.txt
Saved segmentation for GX010152_36_378-jpg_gamma_jpg.rf.9acbfb13ae54b206727a686e4c1c7953.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010152_36_378-jpg_gamma_jpg_segmentations.txt
Found matching YOLO file for GX010155_18_219-jpg_gamma_jpg.rf.e21d8d7715582bd1e9251d064a852c3f.jpg

0: 1024x1024 3799.9ms
Speed: 20.1ms preprocess, 3799.9ms inference, 5.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\segment\pred

  5%|▌         | 3/60 [00:09<02:50,  3.00s/it]

...

Saved segmentation for GX010178_107_2118-jpg_gamma_jpg.rf.0379cd6e1f2ba5c7147c5cf579f8d0b4.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010178_107_2118-jpg_gamma_jpg_segmentations.txt
Saved segmentation for GX010178_107_2118-jpg_gamma_jpg.rf.0379cd6e1f2ba5c7147c5cf579f8d0b4.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010178_107_2118-jpg_gamma_jpg_segmentations.txt
Found matching YOLO file for GX010178_189_3987-jpg_gamma_jpg.rf.2fba5bdda4b891a76f25ad8125c34a2b.jpg

0: 1024x1024 5945.4ms
Speed: 18.7ms preprocess, 5945.4ms inference, 2.6ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\segment\predict58


 45%|████▌     | 27/60 [01:18<01:42,  3.12s/it]

Saved segmentation for GX010178_189_3987-jpg_gamma_jpg.rf.2fba5bdda4b891a76f25ad8125c34a2b.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010178_189_3987-jpg_gamma_jpg_segmentations.txt
Saved segmentation for GX010178_189_3987-jpg_gamma_jpg.rf.2fba5bdda4b891a76f25ad8125c34a2b.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images\GX010178_189_3987-jpg_gamma_jpg_segmentations.txt
Found matching YOLO file for GX010179_88_1697-jpg_gamma_jpg.rf.0140fa90df16901b168aa17552bed81a.jpg



 47%|████▋     | 28/60 [01:19<01:31,  2.86s/it]


KeyboardInterrupt: 

In [6]:
import os
import cv2
import numpy as np

def draw_filled_segmentations_with_opacity(image, xy_coordinates, color=(0, 255, 0), opacity=0.2):
    """
    Draws and fills the segmentation polygons on the image with a specified opacity.
    xy_coordinates: List of lists where each inner list represents a segmentation polygon 
                    (each polygon is a list of [x, y] pairs).
    color: The color to fill the polygons (in BGR format).
    opacity: A float value between 0 and 1 that determines the transparency of the fill.
    """
    # Create an overlay image to draw on
    overlay = image.copy()
    
    # Draw filled polygons on the overlay
    for polygon in xy_coordinates:
        polygon = np.array(polygon, np.int32)  # Convert to NumPy array of int32
        polygon = polygon.reshape((-1, 1, 2))  # Reshape for OpenCV
        cv2.fillPoly(overlay, [polygon], color=color)  # Fill the polygon on the overlay
    
    # Blend the overlay with the original image
    cv2.addWeighted(overlay, opacity, image, 1 - opacity, 0, image)
    return image

def load_segmentations_from_txt(txt_file):
    """
    Loads segmentation polygons from a .txt file.
    Each line in the file contains space-separated coordinates: x1 y1 x2 y2 ... xn yn.
    """
    segmentations = []
    with open(txt_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))  # Read all floats from the line
            polygon = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]  # Convert into (x, y) pairs
            segmentations.append(polygon)
    return segmentations

# Specify the image and corresponding segmentation file
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized"
image_name = "undistorted_GX010091_5_149.jpg_gamma.jpg"  # Replace with the name of your image
image_path = os.path.join(image_dir, image_name)

core_name = image_name.split('.')[0]

# Path to the corresponding segmentations .txt file
txt_file_path = os.path.join(image_dir, f"{core_name}_segmentations.txt")

# Load the image
image = cv2.imread(image_path)

# Check if the segmentation file exists
if os.path.exists(txt_file_path):
    # Load segmentations from the .txt file
    segmentations = load_segmentations_from_txt(txt_file_path)
    
    # Draw and fill the segmentations on the image
    image_with_segmentations = draw_filled_segmentations_with_opacity(image, segmentations)
    
    # Save or display the result
    output_image_path = os.path.join(image_dir, f"{core_name}_with_segmentations.png")
    cv2.imwrite(output_image_path, image_with_segmentations)
    print(f"Saved image with filled segmentations to {output_image_path}")
    
    # Optionally, display the image (for local testing)
    # cv2.imshow('Segmented Image', image_with_segmentations)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print(f"No segmentation file found for {image_name}")


Saved image with filled segmentations to C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized\undistorted_GX010091_5_149_with_segmentations.png


# try skeletonization

In [ ]:

image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right_resized"
image_name = "undistorted_GX010067_33_625.jpg_gamma.jpg"  # Replace with the name of your image
image_path = os.path.join(image_dir, image_name)

core_name = image_name.split('.')[0]

# Path to the corresponding segmentations .txt file
txt_file_path = os.path.join(image_dir, f"{core_name}_segmentations.txt") 

# end here

In [9]:
import os

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

# Helper function to extract the core image name
def extract_core_image_name(image_name):
    # Remove everything after the first hyphen
    core_name = image_name.split('-')[0]
    
    # Split the core name by underscores, and capture the first three parts
    parts = core_name.split('_')
    
    # Rebuild the core name using the first three parts
    core_name = '_'.join(parts[:3])
    
    return core_name

# Helper function to find the matching YOLO file
def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None  # Return None if no match is found

# Helper function to rename images
def rename_images(image_dir, yolo_dir):
    for image_name in os.listdir(image_dir):
        if image_name.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, image_name)
            
            # Extract the core name from the image filename
            core_name = extract_core_image_name(image_name)
            
            # Find the matching YOLO file
            yolo_file_path = find_matching_yolo_file(core_name, yolo_dir)
            
            if yolo_file_path:
                # Create the new image name using the core name
                new_image_name = f"undistorted_{core_name}.jpg"
                new_image_path = os.path.join(image_dir, new_image_name)
                
                # Rename the image
                print(f"Renaming {image_path} to {new_image_path}")
                os.rename(image_path, new_image_path)
            else:
                print(f"No matching YOLO file for {image_name}, skipping...")

# Run the renaming process
rename_images(image_dir, yolo_dir)


Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010152_36_378-jpg_gamma_jpg.rf.dde16353cb779b5cc6dabcd884e15547.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010152_36_378.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010155_18_219-jpg_gamma_jpg.rf.9057ffadeef9e786b3936806a3c9f36a.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010155_18_219.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010155_78_1170-jpg_gamma_jpg.rf.e004afd947d9bd621eb3843251abf362.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010155_78_1170.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010157_174_2582

In [ ]:
import os
from PIL import Image

# Define the folders
base_path = "/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace"  # Replace with your actual path
folders = ['car', 'left', 'right']
target_size = (640, 360)

def resize_images(folder_path):
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Open the image
            image_path = os.path.join(folder_path, filename)
            with Image.open(image_path) as img:
                # Resize image
                resized_img = img.resize(target_size, Image.Resampling.LANCZOS)
                # Save the resized image, overwriting the original
                resized_img.save(image_path)
                print(f"Resized {filename}")

# Process each folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    print(f"\nProcessing folder: {folder}")
    resize_images(folder_path)

print("\nResizing complete!")

# predictions

In [2]:
from ultralytics import YOLO
import cv2


model = YOLO("/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/thesisi/best-car.pt")  # load a custom trained model



img_path = r"/Users/gilbenor/Movies/color_change/colored_image.jpg"

img = cv2.imread(img_path)

data_path = r"/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/colored"

model.predict(data_path, save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)





# model.predict("/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/car", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)

# model.predict("//Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/left", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)

# model.predict("/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/right", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)


# model.predict(r"/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/segmented", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.01)

# /Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/processed_images/molt_processed.jpg
# model.val(data="/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/data.yaml", imgsz=(640,360), conf=0.1,split="test")



# # model.predict("/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/car", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)




# model.predict("/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/image processed/good/640360/right", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.5)
# # model.predict("/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/image processed/good/640360/square", save=True,save_txt=True,show_boxes=True, imgsz=(640,360), conf=0.1)


# from ultralytics import YOLO

# # Load the model
# model = YOLO("/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/thesisi/best-all.pt")

# # Create a dictionary of prediction parameters
# pred_args = {
#     'save': True,
#     'save_txt': True,
#     'show_boxes': True,
#     'imgsz': (640, 360),
#     'conf': 0.1,
#     'project': 'runs/predict/square_molt',  # Save results to project/name
#     'name': 'exp',  # Save results to project/name
#     'exist_ok': True , # Existing project/name ok, do not increment
#     'iou': 0.5,
#     'save_conf': True
# }

# # Make prediction with explicit parameters
# results = model.predict(
#     source="/Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/image processed/good/640360/square",
#     **pred_args
# )




WARNING ⚠️ imgsz=[640, 360] must be multiple of max stride 32, updating to [640, 384]
image 1/191 /Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/colored/colored_undistorted_GX010191_100_1250.jpg: 224x384 2 prawn-models, 475.0ms
image 2/191 /Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/colored/colored_undistorted_GX010191_101_1265.jpg: 224x384 6 prawn-models, 496.7ms
image 3/191 /Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/colored/colored_undistorted_GX010191_102_1277.jpg: 224x384 2 prawn-models, 477.0ms
image 4/191 /Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement_paper_images/molt/all molt/undistorted/resized/colored/colored_undistorted_GX010191_103_1289.jpg: 224x384 1 prawn-model, 326.4ms
image 5/191 /Users/gilbenor/Library/CloudStorage/OneDrive-Personal/measurement

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'prawn-model'}
 obb: None
 orig_img: array([[[212, 156,  31],
         [212, 156,  31],
         [212, 156,  31],
         ...,
         [ 58,  58,  34],
         [ 57,  57,  33],
         [ 56,  56,  32]],
 
        [[212, 156,  31],
         [212, 156,  31],
         [212, 156,  31],
         ...,
         [ 57,  57,  33],
         [ 57,  57,  33],
         [ 56,  56,  32]],
 
        [[212, 156,  31],
         [212, 156,  31],
         [212, 156,  31],
         ...,
         [ 60,  58,  34],
         [ 59,  57,  33],
         [ 59,  57,  33]],
 
        ...,
 
        [[212, 156,  31],
         [212, 156,  31],
         [212, 156,  31],
         ...,
         [212, 157,  30],
         [212, 157,  30],
         [212, 157,  28]],
 
        [[212, 156,  31],
         [212, 156,  31],
         [212,

# script for folders

 # current carapace

In [ ]:
"""
Main script for prawn measurement validation using FiftyOne.

This script:
1. Loads and processes prawn images and YOLO keypoint detections
2. Calculates real-world measurements
3. Compares predictions with ground truth
4. Visualizes results in FiftyOne UI
"""

# Import required libraries
import fiftyone as fo
import os
from data_loader import load_data, create_dataset, process_images
from fifty_one.measurements.analysis.utils import parse_pose_estimation, calculate_euclidean_distance, calculate_real_width, extract_identifier_from_gt
import pandas as pd
import sys
from importlib import reload
import os

# Reload modules for development
reload(sys.modules['data_loader'])
reload(sys.modules['utils'])

# Define paths to data files
filtered_data_file_path = r'/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/src/measurement/ImageJ/Filtered_Data.csv'  # Manual measurements
metadata_file_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/test images.xlsx"        # Camera setup metadata

# Test dataset paths for different ponds
right_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/right'        # Right pond images
right_prediction_folder_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict53/labels"  # YOLO predictions
right_ground_truth_folder_path = r"/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels"  # Ground truth

left_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/left'          # Left pond images
left_prediction_folder_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict53/labels"
left_ground_truth_folder_path = r"/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels"

car_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/car'            # Car pond images
car_prediction_folder_path = r'/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict53/labels'
car_ground_truth_folder_path = r'/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels'

# Load data and create dataset
filtered_df, metadata_df = load_data(filtered_data_file_path, metadata_file_path)
dataset = create_dataset()

# Process each test set (right pond, left pond, car pond)
for folder_path, prediction_folder_path, ground_truth_folder_path in zip(
    [right_folder_path, left_folder_path, car_folder_path],
    [right_prediction_folder_path, left_prediction_folder_path, car_prediction_folder_path],
    [right_ground_truth_folder_path, left_ground_truth_folder_path, car_ground_truth_folder_path]
):
    # Get file paths
    image_paths = [
        os.path.join(folder_path, img) 
        for img in os.listdir(folder_path) 
        if img.endswith(('.png', '.jpg', '.jpeg'))
    ]
    
    ground_truth_paths = [
        os.path.join(ground_truth_folder_path, txt) 
        for txt in os.listdir(ground_truth_folder_path) 
        if txt.endswith('.txt')
    ]
    
    prediction_paths = [
        os.path.join(prediction_folder_path, txt) 
        for txt in os.listdir(prediction_folder_path) 
        if txt.endswith('.txt')
    ]
    
    # Extract pond type from path
    pond_tag = folder_path.split('/')[-1]  # e.g., 'test-right'
    print(pond_tag)
    
    # Process images for current pond
    process_images(
        image_paths=image_paths,
        ground_truth_paths_text=ground_truth_paths,
        prediction_folder_path=prediction_folder_path,
        filtered_df=filtered_df,
        metadata_df=metadata_df,
        dataset=dataset,
        pond_type=pond_tag
    )

# Evaluate keypoint detection performance
results = dataset.evaluate_detections(
    "keypoints",              # Predictions field
    gt_field="keypoints_truth",  # Ground truth field
    eval_key="pose_eval",
    method="coco",           # COCO evaluation metrics
    compute_mAP=True,        # Calculate mean Average Precision
    iou=0.5,                 # IoU threshold
    use_keypoints=True,
    classes=["start-carapace", "eyes"]  # Keypoints to evaluate
)

# Load updated measurements
filtered_df = pd.read_excel('Updated_Filtered_Data_with_real_length.xlsx')

# Add IoU scores to filtered_df
for sample in dataset:
    for keypoint in sample.keypoints.keypoints:
        if hasattr(keypoint, 'pose_eval_iou'):
            filtered_df.loc[
                (filtered_df['id'] == keypoint.id), 
                'pose_eval_iou'
            ] = keypoint.pose_eval_iou

# Save results
filtered_df.to_excel('updated_filtered_data_with_lengths-kalkar.xlsx', index=False)

# Launch FiftyOne UI for visualization
session = fo.launch_app(dataset, port=5153)





right


100%|██████████| 37/37 [00:17<00:00,  2.13it/s]


left


100%|██████████| 5/5 [00:02<00:00,  2.13it/s]


car


...

 76%|███████▌  | 22/29 [00:06<00:01,  3.87it/s]

No ground truth found for undistorted_GX010177_232_3047
No ground truth found for undistorted_GX010178_172_3604.jpg_gamma


100%|██████████| 29/29 [00:08<00:00,  3.49it/s]

No ground truth found for undistorted_GX010179_200_3927.jpg_gamma
Ignoring unsupported parameters {'use_keypoints'} for <class 'fiftyone.utils.eval.coco.COCOEvaluationConfig'>


Evaluating detections...
 100% |███████████████████| 62/62 [1.4s elapsed, 0s remaining, 47.6 samples/s]         
Performing IoU sweep...
 100% |███████████████████| 62/62 [1.1s elapsed, 0s remaining, 56.4 samples/s]         


# body main real

In [2]:
"""
Main script for prawn measurement validation using FiftyOne.for body

This script:
1. Loads and processes prawn images and YOLO keypoint detections
2. Calculates real-world measurements
3. Compares predictions with ground truth
4. Visualizes results in FiftyOne UI
"""
from importlib import reload
import sys
# First import the modules
import sys
from data_loader import *
from fifty_one.measurements.analysis.utils import *
reload(sys.modules['data_loader'])
reload(sys.modules['utils'])
# Import required libraries
import fiftyone as fo
import os
from data_loader import load_data, create_dataset_body, process_images,load_data_body,process_images_body
from fifty_one.measurements.analysis.utils import parse_pose_estimation, calculate_euclidean_distance, calculate_real_width, extract_identifier_from_gt
import pandas as pd
import sys
from importlib import reload
import os

filtered_df = pd.read_excel('/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/src/measurement/ImageJ/final_full_statistics_with_prawn_ids_and_uncertainty - Copy.xlsx')
# Define paths to data files
filtered_data_file_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/src/measurement/ImageJ/final_full_statistics_with_prawn_ids_and_uncertainty - Copy.xlsx"
metadata_file_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/test images.xlsx"        # Camera setup metadata

# Test dataset paths for different ponds
right_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/right'        # Right pond images
right_prediction_folder_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict54/labels"  # YOLO predictions
right_ground_truth_folder_path = r"/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels"  # Ground truth

left_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/left'          # Left pond images
left_prediction_folder_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict54/labels"
left_ground_truth_folder_path = r"/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels"

car_folder_path = r'/Users/gilbenor/Library/CloudStorage/OneDrive-post.bgu.ac.il/measurements/carapace_resized/car'            # Car pond images
car_prediction_folder_path = r'/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/runs/pose/predict54/labels'
car_ground_truth_folder_path = r'/Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/all/labels'

# Load data and create dataset
filtered_df, metadata_df = load_data_body(filtered_data_file_path, metadata_file_path)
dataset = create_dataset_body()

# Process each test set (right pond, left pond, car pond)
for folder_path, prediction_folder_path, ground_truth_folder_path in zip(
    [right_folder_path, left_folder_path, car_folder_path],
    [right_prediction_folder_path, left_prediction_folder_path, car_prediction_folder_path],
    [right_ground_truth_folder_path, left_ground_truth_folder_path, car_ground_truth_folder_path]
):
    # Get file paths
    image_paths = [
        os.path.join(folder_path, img) 
        for img in os.listdir(folder_path) 
        if img.endswith(('.png', '.jpg', '.jpeg'))
    ]
    
    ground_truth_paths = [
        os.path.join(ground_truth_folder_path, txt) 
        for txt in os.listdir(ground_truth_folder_path) 
        if txt.endswith('.txt')
    ]
    
    prediction_paths = [
        os.path.join(prediction_folder_path, txt) 
        for txt in os.listdir(prediction_folder_path) 
        if txt.endswith('.txt')
    ]
    
    # Extract pond type from path
    pond_tag = folder_path.split('/')[-1]  # e.g., 'test-right'
    print(pond_tag)
    
    # Process images for current pond
    process_images(
        image_paths=image_paths,
        ground_truth_paths_text=ground_truth_paths,
        prediction_folder_path=prediction_folder_path,
        filtered_df=filtered_df,
        metadata_df=metadata_df,
        dataset=dataset,
        pond_type=pond_tag
    )


print("Available fields:", dataset.get_field_schema())





# Evaluate keypoint detection performance
results = dataset.evaluate_detections(
    "keypoints",              # Predictions field
    gt_field="keypoints_truth",  # Ground truth field
    eval_key="pose_eval",
    method="coco",           # COCO evaluation metrics
    compute_mAP=True,        # Calculate mean Average Precision
    iou=0.5,                 # IoU threshold
    use_keypoints=True,
    classes=["tail", "rostrum"]  # Keypoints to evaluate
)

filtered_df=pd.read_excel('/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/Updated_full_body_Filtered_Data_with_real_length.xlsx')


# Add IoU scores to filtered_df
for sample in dataset:
    for keypoint in sample.keypoints.keypoints:
        if hasattr(keypoint, 'pose_eval_iou'):
            filtered_df.loc[
                (filtered_df['id'] == keypoint.id), 
                'pose_eval_iou'
            ] = keypoint.pose_eval_iou

# Save results
filtered_df.to_excel('updated_filtered_data_with_lengths_body-all.xlsx', index=False)

# Launch FiftyOne UI for visualization
session = fo.launch_app(dataset, port=5157)





right


 19%|█▉        | 7/37 [00:02<00:10,  2.88it/s]

No matching data found for GX010085_263_3738


 24%|██▍       | 9/37 [00:03<00:09,  3.05it/s]

No matching data found for GX010069_53_796


...

100%|██████████| 29/29 [00:06<00:00,  4.41it/s]

No ground truth found for undistorted_GX010179_200_3927.jpg_gamma
Available fields: OrderedDict([('id', <fiftyone.core.fields.ObjectIdField object at 0x327489130>), ('filepath', <fiftyone.core.fields.StringField object at 0x1062bd250>), ('tags', <fiftyone.core.fields.ListField object at 0x32734e220>), ('metadata', <fiftyone.core.fields.EmbeddedDocumentField object at 0x32734eac0>), ('ground_truth', <fiftyone.core.fields.EmbeddedDocumentField object at 0x32734ea00>), ('detections_predictions', <fiftyone.core.fields.EmbeddedDocumentField object at 0x327373ca0>), ('keypoints', <fiftyone.core.fields.EmbeddedDocumentField object at 0x327373fd0>), ('keypoints_truth', <fiftyone.core.fields.EmbeddedDocumentField object at 0x32735fc70>), ('date', <fiftyone.core.fields.StringField object at 0x1062c21c0>), ('pond', <fiftyone.core.fields.StringField object at 0x32735fca0>), ('avg speed (km/h)', <fiftyone.core.fields.FloatField object at 0x32735f8b0>), ('luminance(cd/m²) ', <fiftyone.core.fields.Fl

Evaluating detections...
 100% |███████████████████| 62/62 [1.4s elapsed, 0s remaining, 44.6 samples/s]         
Performing IoU sweep...
 100% |███████████████████| 62/62 [1.2s elapsed, 0s remaining, 50.2 samples/s]         



Could not connect session, trying again in 10 seconds




Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 s

In [14]:
# Debug file paths and dataset population
import os

# Print a few samples from the dataset
print("\nChecking dataset samples:")
for sample in dataset.take(5):  # Look at first 5 samples
    print(f"\nSample ID: {sample.id}")
    print(f"Filepath exists: {os.path.exists(sample.filepath)}")
    print(f"Filepath: {sample.filepath}")
    print(f"Has detections: {sample.has_field('detections_predictions')}")
    if sample.has_field('detections_predictions'):
        print(f"Number of detections: {len(sample.detections_predictions.detections)}")

# Print dataset stats
print("\nDataset Statistics:")
print(f"Total samples: {len(dataset)}")
print(f"Samples with predictions: {len(dataset.match('detections_predictions'))}")
print(f"Samples with ground truth: {len(dataset.match('ground_truth'))}")

# Check file paths
print("\nChecking image directories:")
for path in [right_folder_path, left_folder_path, car_folder_path]:
    print(f"\nDirectory: {path}")
    print(f"Directory exists: {os.path.exists(path)}")
    if os.path.exists(path):
        files = os.listdir(path)
        print(f"Number of files: {len(files)}")
        print("Sample files:", files[:3])


Checking dataset samples:

Sample ID: 67752cec0bcbd7569c6a1930
Filepath exists: True
Filepath: /Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/right/GX010094_163_2312-jpg_gamma_jpg.rf.84ae6b2ec2ebd66e427f443c4fcb5415.jpg
Has detections: True
Number of detections: 1

Sample ID: 67752cf30bcbd7569c6a1b3f
Filepath exists: True
Filepath: /Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/car/GX010175_215_2644-jpg_gamma_jpg.rf.2c3f24248e75f5b2f5ae1fb0171a2867.jpg
Has detections: True
Number of detections: 1

Sample ID: 67752ce50bcbd7569c6a159b
Filepath exists: True
Filepath: /Users/gilbenor/Downloads/Giant freshwater prawn carapace keypoint detection.v91i.yolov8/right/GX010082_236_3137-jpg_gamma_jpg.rf.0e8d6130cc768fc5c00afc3aa0ee1261.jpg
Has detections: True
Number of detections: 5

Sample ID: 67752cea0bcbd7569c6a1851
Filepath exists: True
Filepath: /Users/gilbenor/Downloads/Giant freshwater prawn carapace keypo

ValueError: Filter must be a ViewExpression or a MongoDB aggregation expression defining a filter; found 'detections_predictions'

In [16]:
fo.app_config.clear_cache()
session = fo.launch_app(dataset, port=5155)

AttributeError: 'AppConfig' object has no attribute 'clear_cache'

In [13]:
print(dataset)

Name:        prawn_combined_dataset
Media type:  image
Num samples: 68
Persistent:  False
Tags:        []
Sample fields:
    id:                     fiftyone.core.fields.ObjectIdField
    filepath:               fiftyone.core.fields.StringField
    tags:                   fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:               fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    detections_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    keypoints:              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    keypoints_truth:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    date:                   fiftyone.core.fields.StringField
    pond:                   fiftyone.core.fields.StringField
    avg speed (km

# body main

In [2]:
import fiftyone as fo
import os
from data_loader import load_data, create_dataset, process_images
from fifty_one.measurements.analysis.utils import calculate_euclidean_distance, calculate_real_width, extract_identifier_from_gt
from data_loader_body import load_data, create_dataset, process_images
import MEC
import skeletonization

import sys
from importlib import reload

# Reload the necessary modules
reload(sys.modules['data_loader_body'])
reload(sys.modules['utils'])
reload(sys.modules['MEC'])
reload(sys.modules['skeletonization'])

# Paths to data files
filtered_data_file_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/src/measurement/ImageJ/final_full_statistics_with_prawn_ids_and_uncertainty - Copy.xlsx"
metadata_file_path = r"/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/test images.xlsx"        # Camera setup metadata

car_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images'
car_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images"

left_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-left\images'
left_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-left\images"

right_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-right\images'
right_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-right\images"

# Load data and metadata
filtered_df, metadata_df = load_data(filtered_data_file_path, metadata_file_path)
# Create the FiftyOne dataset


dataset = create_dataset()


for folder_path, prediction_folder_path in [(car_image_path, car_prediction_folder_path), (left_image_path, left_prediction_folder_path), (right_image_path, right_prediction_folder_path)]:
    image_paths = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
    prediction_paths_text = [os.path.join(prediction_folder_path, txt) for txt in os.listdir(prediction_folder_path) if txt.endswith('.txt')]

    # Process images and predictions using segmentations and bounding boxes
    pond_tag=folder_path.split('\\')[-2]

    if pond_tag=='test-car':
        pond_tag='square-female'
    elif pond_tag=='test-left':
        pond_tag='circular-small'
    else:
        pond_tag='circular-big'

    process_images(image_paths, prediction_folder_path, filtered_df, metadata_df, dataset,pond_tag=pond_tag)

# Start the FiftyOne session
session = fo.launch_app(dataset, port=5153)


  0%|          | 0/30 [00:00<?, ?it/s]

compatible ['GX010152', '36', '378']
compatible ['GX010152', '36', '378']
center [2908.5, 356.5]
center [638.5, 1705.5]
center [3942.0, 2444.0]
center [2908.5, 356.5]
center [638.5, 1705.5]
center [3942.0, 2444.0]


 17%|█▋        | 5/30 [00:00<00:02,  9.43it/s]

compatible ['GX010155', '18', '219']
compatible ['GX010155', '78', '1170']
compatible ['GX010157', '160', '2259']
compatible ['GX010157', '160', '2259']
center [2147.142822265625, 553.1422729492188]
center [4880.0, 1087.0]
compatible ['GX010157', '174', '2582']
compatible ['GX010157', '174', '2582']
center [1555.5, 1884.0]
compatible ['GX010157', '177', '2665']
compatible ['GX010157', '177', '2665']
center [1609.5, 1593.0]
compatible ['GX010157', '68', '883']
compatible ['GX010161', '54', '676']
compatible ['GX010161', '54', '676']
center [3734.5, 1016.0]
compatible ['GX010162', '72', '927']
compatible ['GX010162', '72', '927']
center [2323.5, 1651.0]


 63%|██████▎   | 19/30 [00:00<00:00, 34.79it/s]

...

 92%|█████████▏| 34/37 [00:08<00:00,  7.23it/s]

compatible ['GX010097', '124', '1870']
compatible ['GX010097', '124', '1870']
center [1419.0, 2360.5]
center [1750.5, 946.0]
center [1419.0, 2360.5]
center [1750.5, 946.0]
compatible ['GX010097', '32', '528']
compatible ['GX010097', '32', '528']
center [3510.3232421875, 938.0862426757812]
center [3651.5, 157.5]
center [4369.5, 2551.5]
center [3510.3232421875, 938.0862426757812]
center [3651.5, 157.5]
center [4369.5, 2551.5]
compatible ['GX010097', '60', '1080']
compatible ['GX010097', '60', '1080']
center [3522.5, 2245.0]
center [1904.5, 2178.5]
center [2655.406005859375, 1165.4991455078125]
center [1489.5, 1867.0]
center [4436.0, 2091.0]
center [3522.5, 2245.0]
center [1904.5, 2178.5]
center [2655.406005859375, 1165.4991455078125]
center [1489.5, 1867.0]
center [4436.0, 2091.0]


100%|██████████| 37/37 [00:08<00:00,  4.16it/s]

center [3522.5, 2245.0]
center [1904.5, 2178.5]
center [2655.406005859375, 1165.4991455078125]
center [1489.5, 1867.0]
center [4436.0, 2091.0]
center [3522.5, 2245.0]
center [1904.5, 2178.5]
center [2655.406005859375, 1165.4991455078125]
center [1489.5, 1867.0]
center [4436.0, 2091.0]
compatible ['GX010101', '12', '173']
compatible ['GX010101', '12', '173']
center [3253.0, 995.5]


Processed and saved the updated filtered data.


In [20]:
sample = dataset.first()
print(sample.ground_truth)
print(sample.detections_predictions)

<Detections: {
    'detections': [
        <Detection: {
            'id': '66f26e2bff7cd69554e5adf4',
            'attributes': {},
            'tags': [],
            'label': 'prawn_truth',
            'bounding_box': [
                0.45859375,
                0.01944444444444443,
                0.1828125,
                0.25833333333333336,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
        }>,
        <Detection: {
            'id': '66f26e2bff7cd69554e5adf6',
            'attributes': {},
            'tags': [],
            'label': 'prawn_truth',
            'bounding_box': [
                0.70625,
                0.6361111111111111,
                0.078125,
                0.3638888888888889,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
        }>,
        <Detection: {
            'id': '66f26e2bff7cd69554e5adf8',
            'attributes': {},
            'ta

In [27]:
import fiftyone as fo
import numpy as np

import os
import glob
from PIL import Image
from tqdm import tqdm


# # Function to convert YOLO pose format to FiftyOne format
# def yolo_pose_to_fiftyone(yolo_annotation, image_width, image_height):
#     class_id = int(yolo_annotation[0])
#     bbox_center_x, bbox_center_y, bbox_width, bbox_height = yolo_annotation[1:5]
#     keypoints = yolo_annotation[5:]

#     bbox = [
#         (bbox_center_x - bbox_width/2) * image_width,
#         (bbox_center_y - bbox_height/2) * image_height,
#         bbox_width * image_width,
#         bbox_height * image_height
#     ]

#     fo_keypoints = []
#     for i in range(0, len(keypoints), 3):
#         x, y, visible = keypoints[i:i+3]
#         fo_keypoints.append(fo.Keypoint(
#             points=[(x * image_width, y * image_height)],  # Single point, not a list of points
#             confidence=[float(visible)]  # Single float value
#         ))

#     return fo.Detection(
#         label=str(class_id),
#         bounding_box=bbox,
#         keypoints=fo.Keypoints(keypoints=fo_keypoints)
#     )

from data_loader import process_poses,parse_pose_estimation

# Load your dataset
dataset = fo.Dataset("my_pose_dataset",overwrite=True)
dataset.default_skeleton = fo.KeypointSkeleton(
    labels=["start_carapace", "eyes"],
    edges=[[0, 1]],
)

image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\images"
gt_annotation_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\test version  80\test-car\labels"
pred_annotation_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict85\labels"

for image_path in tqdm(glob.glob(os.path.join(image_dir, "*.jpg"))):
    image_name = os.path.basename(image_path)
    gt_annotation_path = os.path.join(gt_annotation_dir, image_name.replace(".jpg", ".txt"))
    pred_annotation_path = os.path.join(pred_annotation_dir, image_name.replace(".jpg", ".txt"))
    
    sample = fo.Sample(filepath=image_path)
    
    pred_estimations = parse_pose_estimation(pred_annotation_path)
    ground_truths = parse_pose_estimation(gt_annotation_path)

    # Process the pose estimations
    keypoints_list, detections = process_poses(pred_estimations)
    keypoints_list_truth, detections_truth = process_poses(ground_truths, is_ground_truth=True)

    sample["ground_truth"] = fo.Detections(detections=detections_truth)
    sample["detections_predictions"] = fo.Detections(detections=detections)
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints_list)
    sample["keypoints_truth"] = fo.Keypoints(keypoints=keypoints_list_truth)    
    
    dataset.add_sample(sample)

print(f"Total samples in dataset: {len(dataset)}")

# Evaluate detections using the correct field names
results = dataset.evaluate_detections(
    "keypoints",  # Predictions field
    gt_field="keypoints_truth",    # Ground truth field
    eval_key="pose_eval",
    method="coco",
    compute_mAP=True,
    iou=0.5,
    use_keypoints=True,
    classes=["start_carapace", "eyes"]
)

print(results)


100%|██████████| 30/30 [00:00<00:00, 94.68it/s] 


Total samples in dataset: 30
Evaluating detections...
 100% |███████████████████| 30/30 [148.6ms elapsed, 0s remaining, 201.9 samples/s]     
Performing IoU sweep...


c:\Users\gbo10\Videos\research\counting_research_algorithms\.venv\lib\site-packages\fiftyone\utils\iou.py:734: RuntimeWarning: divide by zero encountered in divide
  return np.sum(np.exp(-(dists**2) / (2 * (scale**2)))) / n


 100% |███████████████████| 30/30 [167.2ms elapsed, 0s remaining, 179.5 samples/s]     
{
    "cls": "fiftyone.utils.eval.coco.COCODetectionResults",
    "ytrue": [
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
        "(none)",
       

In [31]:
dataset=fo.load_dataset("my_pose_dataset")

session = fo.launch_app(dataset,port=5153)

In [33]:
for sample in dataset:
    keypoint_ious = []
    for keypoint in sample.keypoints.keypoints:
        if hasattr(keypoint, 'pose_eval_iou'):
            keypoint_ious.append(keypoint.pose_eval_iou)
    
    if keypoint_ious:
        sample['keypoint_max_iou'] = max(keypoint_ious)
        sample['keypoint_avg_iou'] = sum(keypoint_ious) / len(keypoint_ious)
    
    sample.save()

In [25]:
print("Overall Keypoint mAP:", results.mAP())
print("Per-keypoint AP:")
for i, ap in enumerate(results.classwise_AP()):
    print(f"Keypoint {i}: {ap}")

Overall Keypoint mAP: -1
Per-keypoint AP:


AttributeError: 'COCODetectionResults' object has no attribute 'classwise_AP'

In [18]:
print(dataset.bounds("keypoints.keypoints.eval_iou"))


(None, None)
